In [20]:
pip install langchain_community langchain_core

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [21]:
from langchain_groq import ChatGroq

In [22]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0, 
    groq_api_key='', 
    model_name="llama-3.1-70b-versatile"
)

response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [23]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-42713")
page_data=loader.load().pop().page_content
print(page_data)


Apply for Software Engineer

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySoftware EngineerAtlanta, GeorgiaBecome a Part of the NIKE, Inc. TeamNIKE, Inc. does more than outf

In [24]:
pip install langchain_core

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [25]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [26]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Engineer',
 'experience': 'Master’s degree in Computer Engineering, Computer Science, or Computer Analytics and 2 years of experience in the job offered or in an engineer-related occupation',
 'skills': ['Event Driven Architecture',
  'OAuth 2/ Open ID Authentication',
  'Containerization using Docker',
  'Postgres Database',
  'Stateless application development',
  'Google Cloud Platform',
  'Apigee API Management',
  'Terraform',
  'Kubernetes',
  'Python programming',
  'Open Policy Agent',
  'Java',
  'Harness for CI/CD',
  'Behavior Driven Development'],
 'description': 'Develop, code/configure, test programs/systems and solutions problems independently and with minimal supervision in order to meet defined digital product specifications and direction. Develop robust advanced analytics and machine learning solutions that have a direct impact on the business. Advise product owners on discrete technology-related business problems; formulates options, including asse

In [27]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [28]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [29]:
links = collection.query(query_texts=["Experience in Python", "Experience in Python"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}]]

In [31]:
job = json_res
job['skills']

['Event Driven Architecture',
 'OAuth 2/ Open ID Authentication',
 'Containerization using Docker',
 'Postgres Database',
 'Stateless application development',
 'Google Cloud Platform',
 'Apigee API Management',
 'Terraform',
 'Kubernetes',
 'Python programming',
 'Open Policy Agent',
 'Java',
 'Harness for CI/CD',
 'Behavior Driven Development']

In [32]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Engineering Solutions for Your Business Needs

Dear [Client's Name],

I came across your job posting for a Software Engineer and was impressed by the exciting opportunity to develop cutting-edge solutions that drive business impact. As a Business Development Executive at AtliQ, I'd like to introduce our company as a trusted partner in delivering tailored software engineering solutions that meet your specific needs.

At AtliQ, we specialize in harnessing the power of AI and software consulting to streamline business processes, enhance scalability, optimize operations, reduce costs, and boost overall efficiency. Our team of experts has a proven track record of delivering high-quality solutions that align with our clients' goals and objectives.

Our capabilities align perfectly with your job requirements, including:

* Event-Driven Architecture
* OAuth 2/Open ID Authentication
* Containerization using Docker
* Postgres Database
* Stateless application development
